# Exploração e Tratamento dos Dados

### Downloads

In [ ]:
%pip install PyPDF2 nltk

import nltk
nltk.download('stopwords')
nltk.download('rslp')

### Imports

In [8]:
import os
import re
import csv
from PyPDF2 import PdfReader
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.stem import RSLPStemmer

### Extração das questões em PDF
Aqui serão salvos 2 CSVs, um com o enunciado e alternativas juntos e outro com cada um separado.

No segundo caso pode haver algumas aberrações, a maioria das alternativas termina em apenas uma linha, mas uma mesma alternativa pode ter mais de uma linha, o que não foi levado em consideração na lógica do código.

In [9]:
def extract_questions_from_pdf(pdf_path, year):
    reader = PdfReader(pdf_path)
    text = ""
    areas = ["Linguagens", "Ciências Humanas", "Ciências da Natureza", "Matemática"]

    for page in reader.pages:
        text += page.extract_text()

    # Expressão regular para capturar os blocos de questões
    question_blocks = re.split(r'Questão\s+\d+\s*-\s*', text)[1:]  # Dividindo por questão

    questions = []
    questions_full = []

    for block in question_blocks:
        lines = block.splitlines()

        # Extraindo a área de conhecimento
        area_conhecimento = next((word for word in areas if re.search(rf'\b{word}\b', lines[0])), None)

        # Extraindo o enunciado até as alternativas e removendo a área de conhecimento do bloco de texto
        enunciado = ' '.join(lines[1:-5])

        # As alternativas foram consideradas como sendo as ultimas 5 linhas do bloco
        if len(lines) >= 5: 
            questions.append({
                "Ano": year,
                "Enunciado": enunciado.strip(),
                # Nas alternativas são removidas as letras "A" - "E" que estão no começo.
                "Alternativa_A": lines[-5][1:].strip(), 
                "Alternativa_B": lines[-4][1:].strip(),
                "Alternativa_C": lines[-3][1:].strip(),
                "Alternativa_D": lines[-2][1:].strip(),
                "Alternativa_E": lines[-1][1:].strip(),
                "Area_de_Conhecimento": area_conhecimento
            })

        enunciado_alternativas = ' '.join(lines[1:])

        questions_full.append({
                "Ano": year,
                "Enunciado_Alternativas": enunciado_alternativas.strip(),
                "Area_de_Conhecimento": area_conhecimento
            })

    return questions, questions_full

def save_to_csv(questions, csv_path):
    file_exists = os.path.isfile(csv_path)
    keys = questions[0].keys() if questions else []
    
    with open(csv_path, 'a', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        if not file_exists:
            dict_writer.writeheader()
        dict_writer.writerows(questions)

def process_pdfs_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            questions, questions_full= extract_questions_from_pdf(pdf_path, filename[5:9])
            save_to_csv(questions, 'CSV/alternativas_separadas.csv')
            save_to_csv(questions_full, 'CSV/tudo_junto.csv')

folder_path = 'Provas/'

process_pdfs_in_folder(folder_path)

### Extração das questões em TXT

In [10]:
def extract_questions_from_txt(txt_path, area_conhecimento, year):
    with open(txt_path, 'r', encoding='cp1252') as file:
        text = file.read()

    # Expressão regular para capturar os blocos de questões
    question_blocks = re.split(r'QUESTÃO\s+\d+', text)[1:]  # Dividindo por questão
    
    questions = []
    questions_full = []

    for block in question_blocks:
        lines = block.splitlines()

        lines = lines if lines[-1] != '' else lines[:-1]

        # Extraindo o enunciado e alternativas
        enunciado = ' '.join(lines[1:-5]).strip()

        # As alternativas são as últimas 5 linhas do bloco
        if len(lines) >= 5:
            questions.append({
                "Ano": year,
                "Enunciado": enunciado,
                # Nas alternativas são removidas as letras "a." - "e." que estão no começo.
                "Alternativa_A": lines[-5][2:].strip(),
                "Alternativa_B": lines[-4][2:].strip(),
                "Alternativa_C": lines[-3][2:].strip(),
                "Alternativa_D": lines[-2][2:].strip(),
                "Alternativa_E": lines[-1][2:].strip(),
                "Area_de_Conhecimento": area_conhecimento
            })

        enunciado_alternativas = ' '.join(lines).strip()

        questions_full.append({
                "Ano": year,
                "Enunciado_Alternativas": enunciado_alternativas.strip(),
                "Area_de_Conhecimento": area_conhecimento
            })
    
    return questions, questions_full

def save_to_csv(questions, csv_path):
    file_exists = os.path.isfile(csv_path)
    keys = questions[0].keys() if questions else []
    
    with open(csv_path, 'a', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        if not file_exists:
            dict_writer.writeheader()
        dict_writer.writerows(questions)

def process_txts_in_folder(folder_path):
    area_map = {
        "CH": "Ciências Humanas",
        "LC": "Linguagens",
        "CN": "Ciências da Natureza",
        "MT": "Matemática"
    }

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.txt'):
            area_code = filename.split('_')[-1].replace('.txt', '')
            area_conhecimento = area_map.get(area_code, "Área desconhecida")

            txt_path = os.path.join(folder_path, filename)
            questions, questions_full = extract_questions_from_txt(txt_path, area_conhecimento, filename[5:9])
            save_to_csv(questions, 'CSV/alternativas_separadas.csv')
            save_to_csv(questions_full, 'CSV/tudo_junto.csv')

folder_path = 'Provas/'

process_txts_in_folder(folder_path)

### Removendo linhas duplicadas e adicionando "possível imagem" nas células vazias
As provas em PDF de Linguagens foram extraídas duas vezes por conta das questões de espanhol e inglês. Também, como foi feita apenas extração de texto, células vazias muito provavelmente significam imagens.

In [11]:
def remover_duplicatas(csv_path):
    df = pd.read_csv(csv_path)
    df = df.drop_duplicates()
    df.fillna("possível imagem", inplace=True)
    df.to_csv(csv_path, index=False)

csv_path = 'CSV/alternativas_separadas.csv'
remover_duplicatas(csv_path)

csv_path = 'CSV/tudo_junto.csv'
remover_duplicatas(csv_path)

### Pré-processamento de texto
Aqui são aplicadas algumas técnicas de pré-processamento de texto, tais como deixar tudo minúsculo, remover pontuações, fazer tokenização, fazer Stemming, remover stopwords e palavras pequenas mantendo termos importantes.

In [12]:
def preprocess_text(text):
    # Converte todo o texto para letras minúsculas
    text = text.lower()

    # Remove pontuação do texto
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Cria um tokenizador que usa expressões regulares para separar as palavras (tokens)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    stop_words = set(stopwords.words('portuguese'))

    # RSLPStemmer é o stemmer para português
    stemmer = RSLPStemmer()

    # Lista de termos importantes (Linguagens, Ciências Humanas, Ciências da Natureza, Matemática)
    termos_importantes = {
        "literatura", "gramática", "redação", "interpretação", "semântica",
        "figuras", "coesão", "coerência", "gêneros", "sintaxe", "morfologia",
        "poesia", "história", "geografia", "sociologia", "filosofia", "política",
        "economia", "cultura", "revolução", "cidadania", "constituição", "guerra",
        "democracia", "direitos", "biologia", "física", "química", "célula",
        "genética", "evolução", "fotossíntese", "termoquímica", "tabela", "periodica",
        "eletricidade", "força", "energia", "função", "álgebra", "geometria",
        "probabilidade", "estatística", "cálculo", "triângulo", "progressão", "vetor", "polinômio"
    }

    # Filtra os tokens para remover as stopwords, palavras curtas e aplica stemming
    # Considerando que as questões de matemática podem conter só dígitos ou somente um dígitos, essas serão mantidas.
    tokens = [
        stemmer.stem(word) if (word not in termos_importantes and not word.isdigit()) else word 
        for word in tokens 
        if word not in stop_words and (len(word) > 2 or word in termos_importantes or word.isdigit())
    ]

    return ' '.join(tokens)

# Para o DB com as alternativas separadas
df = pd.read_csv('CSV/alternativas_separadas.csv')
for c in df.columns[1:-1]:
    df[c] = df[c].apply(preprocess_text)
df.to_csv('CSV/alternativas_separadas_pp.csv', index=False)

# Para o DB com o enunciado e alternativas juntos
df = pd.read_csv('CSV/tudo_junto.csv')
df['Enunciado_Alternativas'] = df['Enunciado_Alternativas'].apply(preprocess_text)
df.to_csv('CSV/tudo_junto_pp.csv', index=False)